In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from glob import glob
import pandas as pd
import pickle
import random
from PIL import Image
from glob import glob
import re
#from adjustText import adjust_text
import seaborn as sns
import statannot
import argparse
#import torch
import wandb
import sys
sys.path.append('./modified_medsam_repo')

In [14]:
def read_tag_from_wandb(tag, project_name, min_epoch_cutoff = 4):
    # read yolov7 runs
    api = wandb.Api()
    
    # Project is specified by <entity/project-name>
    runs = api.runs(path =f"nair-karthik00/{project_name}", filters = {'tags': tag})
    #runs.append(api.run('nair-karthik00/MedSAM_finetune_hcp_ya_second_round_with_bbox/a935xpc7'))

    summary_list, config_list, name_list = [], [], []
    id_seen = {}
    for run in runs:
        name = run.name
        #if not re.match(pattern, name):
        #    continue
        if run.id in id_seen.keys(): # sometimes wandb passes in the same run twice, so safeguard against this
            continue
        else:
            id_seen[run.id] = True
        
        
        # .summary contains the output keys/values for metrics like accuracy.
        #  We call ._json_dict to omit large files
        dicto = run.summary._json_dict
        #print(dicto)
        if 'val_dice_scores/class_Unknown' not in dicto.keys() or dicto['epoch'] < min_epoch_cutoff: # this run is not finished yet
            print('skip ' + name.split('_')[0].split('label')[1])
            continue
        summary_list.append(dicto)

        # .config contains the hyperparameters.
        #  We remove special values that start with _.
        config_list.append(
            {k: v for k,v in run.config.items()
            if not k.startswith('_')})

        # .name is the human-readable name of the run.
        name_list.append(run.name)


    runs_df = pd.DataFrame({
        "summary": summary_list,
        "config": config_list,
        "name": name_list
        })
    runs_df['label'] = [int(x.split('_')[0].split('label')[1]) for x in runs_df['name']]
    runs_df['val_dice_score'] = [x['val_dice_scores/class_Unknown']['max'] for x in runs_df['summary']]
    runs_df['num_samples_per_epoch'] = [x['num_training_samples'] / (x['epoch']+1) for x in runs_df['summary']]
    runs_df['num_samples_per_epoch'] = runs_df['num_samples_per_epoch'].astype(int)
    runs_df['runtime'] = [x['_runtime'] for x in runs_df['summary']]
    runs_df = runs_df.sort_values('label').reset_index(drop=True)
    ret_df = runs_df[['label', 'val_dice_score', 'num_samples_per_epoch', 'runtime']]

    #assert ret_df.shape[0] == 102
    assert ret_df['label'].value_counts().max() == 1
    
    return ret_df

In [28]:
dfo = read_tag_from_wandb('singletask_30epochs', 'MedSAM_finetune_hcp_ya_second_round_with_bbox', min_epoch_cutoff = 4)
dfo

dfo2 = read_tag_from_wandb('singletask_60epochs', 'MedSAM_finetune_hcp_ya_second_round_with_bbox', min_epoch_cutoff = 4)
#dfo2

skip 90
skip 89
skip 87
skip 86
skip 66
skip 63


In [29]:
print('1: mean time is', dfo['runtime'].mean(), 'seconds')
print('1: total time is', dfo['runtime'].mean() * 102, 'seconds')

print('2: mean time is', dfo2['runtime'].mean(), 'seconds')
print('2: total time is', dfo2['runtime'].mean() * 102, 'seconds')

1: mean time is 9204.95237827301 seconds
1: total time is 938905.142583847 seconds
2: mean time is 8384.323357011333 seconds
2: total time is 855200.982415156 seconds


In [38]:
dfo['num_samples_per_epoch']

0     96532
1     24944
2      6346
3     36553
4     52707
      ...  
93    48913
94    48092
95    37394
96    13605
97    39100
Name: num_samples_per_epoch, Length: 98, dtype: int64

In [39]:
dfo2['num_samples_per_epoch'].sum() * 2 # have to multiply by 2 because the calculation used epochs=30...60 not 0...30

2827072

In [34]:
3085184 * 30

92555520